In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import torch.nn as nn
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 512
EPOCHS = 401
LABELNUMS = 36

In [ ]:
def make_one_hot(data1):
    return (np.arange(LABELNUMS)==data1[:,None]).astype(np.integer)

In [ ]:
train_X = torch.from_numpy(np.load("/Users/denhiroshi/Downloads/model_data/train_Sentense_np.npy").astype("float32"))
train_Y = np.load("/Users/denhiroshi/Downloads/model_data/train_Relation_np.npy").astype("int32")
dev_X = torch.from_numpy(np.load("/Users/denhiroshi/Downloads/model_data/dev_Sentense_np.npy").astype("float32"))
dev_Y = np.load("/Users/denhiroshi/Downloads/model_data/dev_Relation_np.npy").astype("int32")

In [ ]:
train_Y = train_Y.tolist()
dev_Y = dev_Y.tolist()

In [ ]:
# OneHot
train_ontHot_y = []
dev_ontHot_y = []
for trainY in train_Y:
    tmp = np.eye(LABELNUMS)[trainY]
    tmp = tmp.sum(axis = 0).astype("int32")
    tmp = tmp[:-1]
    train_ontHot_y.append(tmp)
train_ontHot_y = torch.Tensor(train_ontHot_y)
print(train_ontHot_y.shape)
for devY in dev_Y:
    tmp = np.eye(LABELNUMS)[devY]
    tmp = tmp.sum(axis = 0).astype("int32")
    tmp = tmp[:-1]
    dev_ontHot_y.append(tmp)
dev_ontHot_y = torch.Tensor(dev_ontHot_y)
print(dev_ontHot_y.shape)

In [ ]:
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
class TEXTCNN(nn.Module):
    def __init__(self):
        super(TEXTCNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, BATCH_SIZE, 200)
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(1,200),
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(50,1)), 
        )
        
        self.conv2 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(3,200), 
            ), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=(48,1)), 
        )
        
        self.conv3 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1, 
                out_channels=16, 
                kernel_size=(5,200), 
                stride = 1,
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(46,1)),
        )
        
        self.output = nn.Linear(48, LABELNUMS-1) 

    def forward(self, x):
        x = x.reshape((-1,1,x.shape[1],x.shape[2]))
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat((x1,x2,x3),1)
        x = x.view(x.size(0), -1)   # 展平
#         x = self.out(x)
        output = self.output(x)
        return output

In [ ]:
model = TEXTCNN()
if torch.cuda.is_available():
    print("To GPU")
    train_X = train_X.cuda()
    train_ontHot_y = train_ontHot_y.cuda()
    dev_X = dev_X.cuda()
    dev_ontHot_y = dev_ontHot_y.cuda()
    model = model.cuda()
else:
    print("Still CPU")

In [ ]:
train = TensorDataset(train_X, train_ontHot_y)
dev = TensorDataset(dev_X, dev_ontHot_y)

In [ ]:
train_loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2, 
)


dev_loader = DataLoader(
    dataset = dev, 
    batch_size = BATCH_SIZE,
    shuffle = True, 
    num_workers = 2, 
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
loss_F = torch.nn.BCELoss()
sigmoid = nn.Sigmoid()
for epoch in range(1,EPOCHS): 
    start = time.time()
    losses = []
    for step,(x,y) in enumerate(train_loader):
        predict = model(x)
        optimizer.zero_grad()
        loss = loss_F(sigmoid(predict), y)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean())
    with torch.no_grad():
        accs = []
        for step,(devx,devy) in enumerate(dev_loader):
            dev_pred = model(devx)
            dev_pred = sigmoid(dev_pred)
            tmp = dev_pred > 0.5
            count = 0
            for i in range(len(tmp.shape[0])):
                if torch.sum(tmp.int()[i] == devy.int()[i]).tolist() == devy.shape[1]:
                    count+=1
            acc = count / devy.shape[0]
            accs.append(acc)
        print(f"{epoch}: accuracy:{sum(accs)/len(accs)} loss: {torch.mean(torch.stack(losses))} time: {time.time() - start}")
    print("flag")
    assert False